In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from device import Device
from loraextensor import LoRaExtensor
from lorawan import LoRaWAN
from loralong import LoRaLong

# Set device consumption characteristics
# https://www.semtech.com/products/wireless-rf/lora-connect/sx1262
sx1262 = Device()
sx1262.SetTxCurrent(118.0)          # 868~915 MHz/+22dBm
sx1262.SetRxCurrent(5.3)            # DCDC Mode/Rx Boosted, LoRa, 125Hz
sx1262.SetIdleCurrent(0.6)          # RC mode/XOSC off
sx1262.SetSleepCurrent(0.0012)      # Sleep with warm start/RC64k
sx1262.SetVoltage(3.3)

generateImages = 0

sf = 10
bw = 125
cr = 1
pkt = 25

In [ ]:
h = 1
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_TxCurr_array
eixo_y2 = ry_long_TxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')


# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(a)', fontsize=20)
ax.set_ylabel('TX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='upper center', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_txXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
h = 6
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_TxCurr_array
eixo_y2 = ry_long_TxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')


# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(b)', fontsize=20)
ax.set_ylabel('TX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='upper center', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_txXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
h = 12
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_TxCurr_array
eixo_y2 = ry_long_TxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')


# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(c)', fontsize=20)
ax.set_ylabel('TX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='upper center', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_txXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
h = 1
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_RxCurr_array
eixo_y2 = ry_long_RxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')

# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(a)', fontsize=20)
ax.set_ylabel('RX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

# for x_m in N:
#     ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
#     y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
#     y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
#     ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
#     ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
#     ax.text(x_m, y1_m, f'{y1_m:.0f}%', fontsize=16, ha='left', va='bottom', color='black')
#     ax.text(x_m, y2_m, f'{y2_m:.0f}%', fontsize=16, ha='left', va='bottom', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.tight_layout()
plt.legend(loc='lower center', ncol=2)
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_rxXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
h = 6
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_RxCurr_array
eixo_y2 = ry_long_RxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')

# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(a)', fontsize=20)
ax.set_ylabel('RX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

# for x_m in N:
#     ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
#     y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
#     y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
#     ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
#     ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
#     ax.text(x_m, y1_m, f'{y1_m:.0f}%', fontsize=16, ha='left', va='bottom', color='black')
#     ax.text(x_m, y2_m, f'{y2_m:.0f}%', fontsize=16, ha='left', va='bottom', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.tight_layout()
plt.legend(loc='lower center', ncol=2)
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_rxXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
h = 12
sf = 10

perS = 60*60*h
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt + 8
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

N = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

for n in N:

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(perS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(perS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600))*1000)

eixo_x = np.arange(1,17)

eixo_y1 = ry_lorawan_RxCurr_array
eixo_y2 = ry_long_RxCurr_array
# Largura de cada barra
largura = 0.25

fig, ax = plt.subplots(figsize=(9, 6))
ax.bar(eixo_x - largura/2, eixo_y2, color='red', linewidth=largura, \
        label=f'Long Preamble')
ax.bar(eixo_x + largura/2, eixo_y1, color='black', linewidth=largura, \
        label=f'LoRaWAN')

# ax.set_title('Relay Consumption x App Uplink Periodicity', fontsize=20, weight='bold')
ax.set_xlabel('Endpoints Connected (N) \n(a)', fontsize=20)
ax.set_ylabel('RX Consumption %', fontsize=20)
ax.set_ylim(0, 100)
ax.set_xlim(0, 17)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%3.0f'))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == 0 or x == 17 else f'{int(x)}'))

# for x_m in N:
#     ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
#     y1_m = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
#     y2_m = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
#     ax.plot(x_m, y1_m, marker='.', color='black', markersize=6)
#     ax.plot(x_m, y2_m, marker='.', color='red', markersize=6)
#     ax.text(x_m, y1_m, f'{y1_m:.0f}%', fontsize=16, ha='left', va='bottom', color='black')
#     ax.text(x_m, y2_m, f'{y2_m:.0f}%', fontsize=16, ha='left', va='bottom', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.tight_layout()
plt.legend(loc='lower center', ncol=2)
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_rxXn_{h:0.0f}h.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()

In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 7
bw = 125
cr = 1
pkt = 25

n = 1

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(a)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(10, 10000)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()


In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 10
bw = 125
cr = 1
pkt = 25

n = 1

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(b)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(10, 10000)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()


In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 12
bw = 125
cr = 1
pkt = 25

n = 1

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(c)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(10, 10000)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.1f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()


In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 10
bw = 125
cr = 1
pkt = 25

n = 1

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(a)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(650, 1200)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()


In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 10
bw = 125
cr = 1
pkt = 25

n = 8

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(b)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(650, 1200)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()


In [ ]:
simDurationH = 24*365*10
simDuration = simDurationH * 60*60

sf = 10
bw = 125
cr = 1
pkt = 25

n = 16

sf_wan = sf
bw_wan = bw
cr_wan = cr
pkt_wan = pkt
ry_lorawan = LoRaWAN(pkt_wan, sf_wan, bw_wan, cr_wan)
# ry_lorawan.n_accuracy = 10

sf_lng = sf
bw_lng = bw
cr_lng = cr
pkt_lng = pkt
ry_long = LoRaLong(pkt_lng, sf_lng, bw_lng, cr_lng)

print("loRa Extensor Comparison - relay uplink simulation")
print("Device Consumption x Application Uplink Periodicity")
print("Simulation Duration [y]:", simDurationH/(24*365))

ry_lorawan_TxTime_array = []
ry_lorawan_RxTime_array = []
ry_lorawan_SleepTime_array = []
ry_lorawan_TxCurr_array = []
ry_lorawan_RxCurr_array = []
ry_lorawan_SleepCurr_array = []
ry_lorawan_TotalMeanCon_array = []

ry_long_TxTime_array = []
ry_long_RxTime_array = []
ry_long_SleepTime_array = []
ry_long_TxCurr_array = []
ry_long_RxCurr_array = []
ry_long_SleepCurr_array = []
ry_long_TotalMeanCon_array = []

periodicityS = 86400
periodicityS_array = np.linspace(3600, periodicityS, periodicityS-1)
periodicityMin_array = []
periodicityH_array = []
for pS in periodicityS_array:

    periodicityMin_array.append(pS/60)
    periodicityH_array.append(pS/3600)

    txTime, rxTime, idleTime, sleepTime = ry_lorawan.SimulateRelayConfirmedLoRaWAN(pS, pkt_wan, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_lorawan_TxTime_array.append((txTime/totalTime)*100)
    ry_lorawan_RxTime_array.append((rxTime/totalTime)*100)
    ry_lorawan_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_lorawan_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_lorawan_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_lorawan_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_lorawan_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))

    txTime, rxTime, idleTime, sleepTime = ry_long.SimulateRelayLoRaLong(pS, pkt_lng, simDuration, n)
    totalTime = txTime + rxTime + idleTime + sleepTime
    ry_long_TxTime_array.append((txTime/totalTime)*100)
    ry_long_RxTime_array.append((rxTime/totalTime)*100)
    ry_long_SleepTime_array.append((sleepTime/totalTime)*100)
    total_mAh, tx_mAh, rx_mAh, idle_mAh, sleep_mAh = sx1262.GetAppConsumption(totalTime, txTime, rxTime, idleTime, sleepTime)
    ry_long_TxCurr_array.append((tx_mAh/total_mAh)*100)
    ry_long_RxCurr_array.append((rx_mAh/total_mAh)*100)
    ry_long_SleepCurr_array.append((sleep_mAh/total_mAh)*100)
    ry_long_TotalMeanCon_array.append((total_mAh/(totalTime/3600)*1000))


eixo_x = periodicityH_array
eixo_y1 = ry_lorawan_TotalMeanCon_array
eixo_y2 = ry_long_TotalMeanCon_array

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(eixo_x, eixo_y1, color='black', linewidth=2, \
        label=f'LoRaWAN')
ax.plot(eixo_x, eixo_y2, color='red', linewidth=2, \
        label=f'Long Preamble')
ax.set_xlabel('Uplink Periodicity (Hours) \n(c)', fontsize=20)
ax.set_ylabel('Mean Current (uA)', fontsize=20)
ax.set_yscale('log')
ax.set_ylim(650, 1200)
ax.set_xlim(-1, 24)
ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.0f'))
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: '' if x == -1 else f'{int(x)}'))

x_marcar = [1, 6, 12, 18]
for x_m in x_marcar:
    m_y1 = np.interp(x_m, eixo_x, eixo_y1)  # estima o y naquele x
    m_y2 = np.interp(x_m, eixo_x, eixo_y2)  # estima o y naquele x
    ax.axvline(x=0, color='black', linestyle='-', linewidth=1)
    ax.plot(x_m, m_y1, marker='o', color='black', markersize=6)
    ax.text(x_m, m_y1, f'{(m_y1):.0f}uA', fontsize=16, ha='left', va='bottom', color='black')
    ax.plot(x_m, m_y2, marker='o', color='red', markersize=6)
    ax.text(x_m, m_y2, f'{(m_y2):.0f}uA', fontsize=16, ha='left', va='top', color='red')

ax.grid(True, which='both', linestyle=':', linewidth=1.5)
plt.legend(loc='lower right', ncol=2)
plt.tight_layout()
# generateImages = 1
if generateImages == 1:
    plt.savefig(f'rl_meanCurrXappPer_BW{bw:0.0f}_SF{sf:0.0f}_N{n:0.0f}.png', dpi=300, bbox_inches='tight', transparent=False, facecolor='white')
plt.show()
